In [ ]:
pip install alpaca-py

In [2]:
pip install tensorflow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.3/244.3 MB 7.3 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 980.5/980.5 kB 21.4 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 27.0 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 28.0 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 25.0/25.0 MB 20.8 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 34.4 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.5/126.5 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 439.2/439.2 kB 17.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.6/4.6 MB 21.5 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 781.3/781.3 kB 14.8 

In [1]:
pip install -U scikit-learn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 18.5 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.0.2
    Uninstalling scikit-learn-1.0.2:
      Successfully uninstalled scikit-learn-1.0.2

[notice] A new release of pip available: 23.0 -> 23.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [15]:
pip install nest-asyncio


[notice] A new release of pip available: 23.0 -> 23.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [9]:
import nest_asyncio
nest_asyncio.apply()

In [10]:
from alpaca.data.historical import CryptoHistoricalDataClient
from alpaca.data.requests import CryptoBarsRequest
from alpaca.data.timeframe import TimeFrame
from alpaca.trading.client import TradingClient
from alpaca.trading.requests import MarketOrderRequest
from alpaca.trading.enums import OrderSide, TimeInForce
from datetime import datetime
from dateutil.relativedelta import relativedelta
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dense, Dropout, LSTM
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error
import json
import logging
import config
import asyncio

In [11]:
# ENABLE LOGGING - options, DEBUG,INFO, WARNING?
logging.basicConfig(level=logging.INFO,
                    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)


# Alpaca Trading Client
trading_client = TradingClient(
    config.APCA_API_KEY_ID, config.APCA_API_SECRET_KEY, paper=True)


# Trading variables
trading_pair = 'ETH/USD'
qty_to_trade = 5
# Wait time between each bar request and training model
waitTime = 600
data = 0

current_position, current_price = 0, 0
predicted_price = 0

In [12]:
async def main():
    '''
    Main function to get latest asset data and check possible trade conditions
    '''

    while True:
        logger.info('----------------------------------------------------')

        pred = stockPred()
        global predicted_price
        predicted_price = pred.predictModel()
        logger.info("Predicted Price is {0}".format(predicted_price))
        l1 = loop.create_task(check_condition())
        await asyncio.wait([l1])
        await asyncio.sleep(waitTime)

In [13]:
class stockPred:
    def __init__(self,
                 past_days: int = 50,
                 trading_pair: str = 'ETHUSD',
                 exchange: str = 'FTXU',
                 feature: str = 'close',

                 look_back: int = 100,

                 neurons: int = 50,
                 activ_func: str = 'linear',
                 dropout: float = 0.2,
                 loss: str = 'mse',
                 optimizer: str = 'adam',
                 epochs: int = 20,
                 batch_size: int = 32,
                 output_size: int = 1
                 ):
        self.exchange = exchange
        self.feature = feature

        self.look_back = look_back

        self.neurons = neurons
        self.activ_func = activ_func
        self.dropout = dropout
        self.loss = loss
        self.optimizer = optimizer
        self.epochs = epochs
        self.batch_size = batch_size
        self.output_size = output_size

    def getAllData(self):

        # Alpaca Market Data Client
        data_client = CryptoHistoricalDataClient()

        time_diff = datetime.now() - relativedelta(hours=1000)
        logger.info("Getting bar data for {0} starting from {1}".format(
            trading_pair, time_diff))
        # Defining Bar data request parameters
        request_params = CryptoBarsRequest(
            symbol_or_symbols=[trading_pair],
            timeframe=TimeFrame.Hour,
            start=time_diff
        )
        # Get the bar data from Alpaca
        df = data_client.get_crypto_bars(request_params).df
        global current_price
        current_price = df.iloc[-1]['close']
        return df

    def getFeature(self, df):
        data = df.filter([self.feature])
        data = data.values
        return data

    def scaleData(self, data):
        scaler = MinMaxScaler(feature_range=(-1, 1))
        scaled_data = scaler.fit_transform(data)
        return scaled_data, scaler

    # train on all data for which labels are available (train + test from dev)
    def getTrainData(self, scaled_data):
        x, y = [], []
        for price in range(self.look_back, len(scaled_data)):
            x.append(scaled_data[price - self.look_back:price, :])
            y.append(scaled_data[price, :])
        return np.array(x), np.array(y)

    def LSTM_model(self, input_data):
        model = Sequential()
        model.add(LSTM(self.neurons, input_shape=(
            input_data.shape[1], input_data.shape[2]), return_sequences=True))
        model.add(Dropout(self.dropout))
        model.add(LSTM(self.neurons, return_sequences=True))
        model.add(Dropout(self.dropout))
        model.add(LSTM(self.neurons))
        model.add(Dropout(self.dropout))
        model.add(Dense(units=self.output_size))
        model.add(Activation(self.activ_func))

        model.compile(loss=self.loss, optimizer=self.optimizer)
        return model

    def trainModel(self, x, y):

        x_train = x[: len(x) - 1]
        y_train = y[: len(x) - 1]
        model = self.LSTM_model(x_train)
        modelfit = model.fit(x_train, y_train, epochs=self.epochs,
                             batch_size=self.batch_size, verbose=1, shuffle=True)
        return model, modelfit

    def predictModel(self):
        logger.info("Getting Ethereum Bar Data")
        # get all data
        df = self.getAllData()

        logger.info("Getting Feature: {}".format(self.feature))
        # get feature (closing price)
        data = self.getFeature(df)

        logger.info("Scaling Data")
        # scale data and get scaler
        scaled_data, scaler = self.scaleData(data)

        logger.info("Getting Train Data")
        # get train data
        x_train, y_train = self.getTrainData(scaled_data)

        logger.info("Training Model")
        # Creates and returns a trained model
        model = self.trainModel(x_train, y_train)[0]

        logger.info("Extracting data to predict on")
        # Extract the Closing prices that need to be fed into predict the result
        x_pred = scaled_data[-self.look_back:]
        x_pred = np.reshape(x_pred, (1, x_pred.shape[0]))

        # Predict the result
        logger.info("Predicting Price")
        pred = model.predict(x_pred).squeeze()
        pred = np.array([float(pred)])
        pred = np.reshape(pred, (pred.shape[0], 1))

        # Inverse the scaling to get the actual price
        pred_true = scaler.inverse_transform(pred)
        return pred_true[0][0]

In [14]:
async def check_condition():
    '''
    Strategy:
    - If the predicted price an hour from now is above the current price and we do not have a position, buy
    - If the predicted price an hour from now is below the current price and we do have a position, sell
    '''
    global current_position, current_price, predicted_price
    current_position = get_positions()
    logger.info("Current Price is: {0}".format(current_price))
    logger.info("Current Position is: {0}".format(current_position))
    # If we do not have a position and current price is less than the predicted price place a market buy order
    if float(current_position) <= 0.01 and current_price < predicted_price:
        logger.info("Placing Buy Order")
        buy_order = await post_alpaca_order('buy')
        if buy_order:
            logger.info("Buy Order Placed")

    # If we do have a position and current price is greater than the predicted price place a market sell order
    if float(current_position) >= 0.01 and current_price > predicted_price:
        logger.info("Placing Sell Order")
        sell_order = await post_alpaca_order('sell')
        if sell_order:
            logger.info("Sell Order Placed")

In [15]:
async def post_alpaca_order(side):
    '''
    Post an order to Alpaca
    '''
    try:
        if side == 'buy':
            market_order_data = MarketOrderRequest(
                symbol="ETHUSD",
                qty=qty_to_trade,
                side=OrderSide.BUY,
                time_in_force=TimeInForce.GTC
            )
            buy_order = trading_client.submit_order(
                order_data=market_order_data
            )
            return buy_order
        else:
            market_order_data = MarketOrderRequest(
                symbol="ETHUSD",
                qty=current_position,
                side=OrderSide.SELL,
                time_in_force=TimeInForce.GTC
            )
            sell_order = trading_client.submit_order(
                order_data=market_order_data
            )
            return sell_order
    except Exception as e:
        logger.exception(
            "There was an issue posting order to Alpaca: {0}".format(e))
        return False


def get_positions():
    positions = trading_client.get_all_positions()
    # print(positions[0])
    global current_position
    for p in positions:
        if p.symbol == 'ETHUSD':
            current_position = p.qty
            return current_position
    return current_position

In [ ]:
loop = asyncio.get_event_loop()
loop.run_until_complete(main())
loop.close()

2023-02-21 01:22:07,668 - __main__ - INFO - ----------------------------------------------------
2023-02-21 01:22:07,669 - __main__ - INFO - Getting Ethereum Bar Data
2023-02-21 01:22:07,671 - __main__ - INFO - Getting bar data for ETH/USD starting from 2023-01-10 09:22:07.671310
2023-02-21 01:22:09,341 - __main__ - INFO - Getting Feature: close
2023-02-21 01:22:09,342 - __main__ - INFO - Scaling Data
2023-02-21 01:22:09,343 - __main__ - INFO - Getting Train Data
2023-02-21 01:22:09,345 - __main__ - INFO - Training Model


Epoch 1/20
28/28 [==============================] - 4s 69ms/step - loss: 0.0535
Epoch 2/20
28/28 [==============================] - 2s 68ms/step - loss: 0.0193
Epoch 3/20
28/28 [==============================] - 2s 68ms/step - loss: 0.0174
Epoch 4/20
28/28 [==============================] - 2s 68ms/step - loss: 0.0142
Epoch 5/20
28/28 [==============================] - 2s 68ms/step - loss: 0.0134
Epoch 6/20
28/28 [==============================] - 2s 68ms/step - loss: 0.0124
Epoch 7/20
28/28 [==============================] - 2s 68ms/step - loss: 0.0115
Epoch 8/20
28/28 [==============================] - 2s 69ms/step - loss: 0.0106
Epoch 9/20
28/28 [==============================] - 2s 70ms/step - loss: 0.0106
Epoch 10/20
28/28 [==============================] - 2s 69ms/step - loss: 0.0100
Epoch 11/20
28/28 [==============================] - 2s 68ms/step - loss: 0.0091
Epoch 12/20
28/28 [==============================] - 2s 69ms/step - loss: 0.0089
Epoch 13/20
28/28 [==================

2023-02-21 01:22:50,661 - __main__ - INFO - Extracting data to predict on
2023-02-21 01:22:50,661 - __main__ - INFO - Predicting Price


1/1 [==============================] - 1s 580ms/step


2023-02-21 01:22:51,263 - __main__ - INFO - Predicted Price is 1701.8584552285079
2023-02-21 01:22:52,089 - __main__ - INFO - Current Price is: 1696.29
2023-02-21 01:22:52,091 - __main__ - INFO - Current Position is: 0
2023-02-21 01:22:52,091 - __main__ - INFO - Placing Buy Order
2023-02-21 01:22:52,325 - __main__ - INFO - Buy Order Placed
2023-02-21 01:32:52,307 - __main__ - INFO - ----------------------------------------------------
2023-02-21 01:32:52,308 - __main__ - INFO - Getting Ethereum Bar Data
2023-02-21 01:32:52,310 - __main__ - INFO - Getting bar data for ETH/USD starting from 2023-01-10 09:32:52.309920
2023-02-21 01:32:53,696 - __main__ - INFO - Getting Feature: close
2023-02-21 01:32:53,697 - __main__ - INFO - Scaling Data
2023-02-21 01:32:53,699 - __main__ - INFO - Getting Train Data
2023-02-21 01:32:53,701 - __main__ - INFO - Training Model


...

2023-02-21 01:44:28,910 - __main__ - INFO - Predicted Price is 1691.235806603432
2023-02-21 01:44:29,605 - __main__ - INFO - Current Price is: 1700.04
2023-02-21 01:44:29,606 - __main__ - INFO - Current Position is: 4.987499999
2023-02-21 01:44:29,606 - __main__ - INFO - Placing Sell Order
2023-02-21 01:44:29,929 - __main__ - ERROR - There was an issue posting order to Alpaca: {"available":"0","balance":"0","code":40310000,"message":"insufficient balance for ETH (requested: 4.987499999, available: 0)","symbol":"USD"}
Traceback (most recent call last):
  File "/Users/arnavgupta/opt/anaconda3/lib/python3.9/site-packages/alpaca/common/rest.py", line 187, in _one_request
    response.raise_for_status()
  File "/Users/arnavgupta/opt/anaconda3/lib/python3.9/site-packages/requests/models.py", line 1021, in raise_for_status
    raise HTTPError(http_error_msg, response=self)
requests.exceptions.HTTPError: 403 Client Error: Forbidden for url: https://paper-api.alpaca.markets/v2/orders

During ha

Epoch 1/20
28/28 [==============================] - 4s 68ms/step - loss: 0.0579
Epoch 2/20
28/28 [==============================] - 2s 67ms/step - loss: 0.0185
Epoch 3/20
28/28 [==============================] - 2s 68ms/step - loss: 0.0157
Epoch 4/20
28/28 [==============================] - 2s 67ms/step - loss: 0.0135
Epoch 5/20
28/28 [==============================] - 2s 67ms/step - loss: 0.0121
Epoch 6/20
28/28 [==============================] - 2s 67ms/step - loss: 0.0121
Epoch 7/20
28/28 [==============================] - 2s 68ms/step - loss: 0.0112
Epoch 8/20
28/28 [==============================] - 2s 68ms/step - loss: 0.0105
Epoch 9/20
28/28 [==============================] - 2s 68ms/step - loss: 0.0100
Epoch 10/20
28/28 [==============================] - 2s 71ms/step - loss: 0.0114
Epoch 11/20
28/28 [==============================] - 2s 68ms/step - loss: 0.0092
Epoch 12/20
28/28 [==============================] - 2s 68ms/step - loss: 0.0093
Epoch 13/20
28/28 [==================

2023-02-21 01:55:12,903 - __main__ - INFO - Extracting data to predict on
2023-02-21 01:55:12,903 - __main__ - INFO - Predicting Price


1/1 [==============================] - 1s 559ms/step


2023-02-21 01:55:13,480 - __main__ - INFO - Predicted Price is 1698.9512022671106
2023-02-21 01:55:14,236 - __main__ - INFO - Current Price is: 1698.51
2023-02-21 01:55:14,237 - __main__ - INFO - Current Position is: 4.987499999
